# GitHub repositories analysis

In [2]:
import pandas as pd
import os
import glob

## Read the data

In [3]:
files = glob.glob("./data/**/*.csv", recursive=True)
print("Encontrados", len(files), "ficheros")

Encontrados 16 ficheros


In [4]:
df = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

In [11]:
df.sort_values(by=["ts", "views_total"], ascending=False).head(3)

,Unnamed: 0,name,ts,stars_count,views_total,views_uniques,clones_total,clones_uniques
43,3,ml-github,2022-07-13,0,6,1,4,3
41,1,golang-rest-api-cassandra,2022-07-13,2,2,1,42,32
40,0,b0rr3g0-blog,2022-07-13,0,0,0,1,1


## Data analysis

### Views

In [5]:
views = df.groupby(['name']).agg({"views_total": ["sum"]})
views = views[views[('views_total', 'sum')] > 0]
views.sort_values(by=[('views_total', 'sum')], ascending=False)

,views_total
,sum
name,
quarkus-kotlin-example,177
golang-rest-api-cassandra,91
kotlin-ktor,84
kustomize-vs-helm,43
ta,38
ml-github,35
pelorus-workshop,34
library,28


### Clones

In [6]:
clones = df.groupby(['name']).agg({"clones_uniques": ["sum"]})
clones = clones[clones[('clones_uniques', 'sum')] > 0]
clones.sort_values(by=[('clones_uniques', 'sum')], ascending=False)

,clones_uniques
,sum
name,
ta,320
golang-rest-api-cassandra,297
mqtt-golang-influxdb,149
iot-devices-crud,130
pelorus-workshop,14
ml-github,13
b0rr3g0-blog,10
service-mesh-wasm-go,5


# Stars

In [7]:
stars = df.groupby(['name']).agg({"stars_count": ["max"]})
stars = stars[stars[('stars_count', 'max')] > 0]
stars.sort_values(by=[('stars_count', 'max')], ascending=False)

,stars_count
,max
name,
mqtt-golang-influxdb,9
spring-kotlin-crud,6
kotlin-ktor,5
kops-aws,4
kustomize-vs-helm,3
ta,3
golang-k8s-helm-helloworld,2
golang-rest-api-cassandra,2
